In [2]:
import torch
import matplotlib.pyplot as plt
import torch
from torch.optim import LBFGS
import time
import numpy as np
import os
import torch.nn as nn
from torch.autograd import grad

device = torch.device('cuda')  # Usamos CPU para este ejemplo

In [3]:
# Definir la función legendre de forma vectorizada
def legendre(n, x, lb, ub):
    x = 2 * (x - lb) / (ub - lb) - 1    
    P0 = torch.ones_like(x)
    if n == 0:
        return P0
    P1 = x
    if n == 1:
        return P1
    for _ in range(2, n + 1):
        P0, P1 = P1, ((2 * _ - 1) * x * P1 - (_ - 1) * P0) / _
    return P1
 
 

 
#Evaluación de la serie de Legendre 2-D utilizando broadcasting
def evaluate_legendre_series(coefficients, leg_x, leg_y):
    n = int(torch.sqrt(torch.tensor(coefficients.numel()).float()))  # Convert to tensor before sqrt
    coefficients = coefficients.view(n, n)
    # Utilizando broadcasting para calcular la serie de Legendre
    result = torch.sum(coefficients[:, :, None, None] * leg_x[:, None, :, :] * leg_y[None, :, :, :], dim=(0, 1))
    return result 

# Function to compute derivatives
def derivative(dy: torch.Tensor, x: torch.Tensor, order: int = 1) -> torch.Tensor:
    for i in range(order):
        dy = torch.autograd.grad(
            dy, x, grad_outputs=torch.ones_like(dy), create_graph=True, retain_graph=True
        )[0]
    return dy

In [4]:
# Precomputar leg_x y leg_y una vez
start_precompute = time.time()

# Elige una suposición inicial para los coeficientes
N = 50  # Grado máximo de los polinomios de Legendre
initial_guess = torch.zeros(N*N, requires_grad=True, dtype=torch.float64, device=device)  # Suposición inicial para los coeficientes
lb, ub = 0, 1  # Límites de integración
# Datos de entrada para la función y los polinomios de Legendre
X1, X2 = torch.meshgrid(torch.linspace(0, 1, 100, dtype=torch.float64, device=device), torch.linspace(0, 1, 100, dtype=torch.float64, device=device))
# Make X1 and X2 require gradients
X1.requires_grad_()
X2.requires_grad_()
n = int(torch.sqrt(torch.tensor(initial_guess.numel(), device=device)))
leg_x = torch.stack([legendre(i, X1, lb, ub).to(device).double() for i in range(n)], dim=0)
leg_y = torch.stack([legendre(j, X2, lb, ub).to(device).double() for j in range(n)], dim=0)

end_precompute = time.time()
time_precompute = end_precompute - start_precompute
print(f"Tiempo de precomputación de Legendre polynomials: {time_precompute:.4f} segundos")

/home/oscar-rincon/miniconda3/envs/ReScience-PINNs-env/lib/python3.12/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1716578970623/work/aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Tiempo de precomputación de Legendre polynomials: 0.5486 segundos


In [5]:
# Definir la función legendre de forma vectorizada
def legendre(n, x, lb, ub):
    x = 2 * (x - lb) / (ub - lb) - 1    
    P0 = torch.ones_like(x)
    if n == 0:
        return P0
    P1 = x
    if n == 1:
        return P1
    for _ in range(2, n + 1):
        P0, P1 = P1, ((2 * _ - 1) * x * P1 - (_ - 1) * P0) / _
    return P1
 
 

 
#Evaluación de la serie de Legendre 2-D utilizando broadcasting
def evaluate_legendre_series(coefficients, leg_x, leg_y):
    n = int(torch.sqrt(torch.tensor(coefficients.numel()).float()))  # Convert to tensor before sqrt
    coefficients = coefficients.view(n, n)
    # Utilizando broadcasting para calcular la serie de Legendre
    result = torch.sum(coefficients[:, :, None, None] * leg_x[:, None, :, :] * leg_y[None, :, :, :], dim=(0, 1))
    return result 
 
# Precomputar leg_x y leg_y una vez
start_precompute = time.time()

# Elige una suposición inicial para los coeficientes
N = 50  # Grado máximo de los polinomios de Legendre
initial_guess = torch.zeros(N*N, requires_grad=True, dtype=torch.float64, device=device)  # Suposición inicial para los coeficientes
lb, ub = 0, 1  # Límites de integración
# Datos de entrada para la función y los polinomios de Legendre
x1,x2 = torch.linspace(0, 1, 100, dtype=torch.float64, device=device).requires_grad_(), torch.linspace(0, 1, 100, dtype=torch.float64, device=device).requires_grad_()
X1, X2 = torch.meshgrid(x1,x2)
n = int(torch.sqrt(torch.tensor(initial_guess.numel(), device=device)))
leg_x = torch.stack([legendre(i, X1, lb, ub).to(device).double() for i in range(n)], dim=0)
leg_y = torch.stack([legendre(j, X2, lb, ub).to(device).double() for j in range(n)], dim=0)

end_precompute = time.time()
time_precompute = end_precompute - start_precompute
print(f"Tiempo de precomputación de Legendre polynomials: {time_precompute:.4f} segundos") 
 
 
criterion = nn.MSELoss()
# Define la función de error
# Definir la función de error
def error_function(coefficients, leg_x, leg_y):
    N_xy_p = evaluate_legendre_series(coefficients, leg_x, leg_y)
    u_t = x1 * x2 * (x1 - 1) * (x2 - 1) * N_xy_p
    u_xx = derivative(u_t, x1, order=2) 
    u_yy = derivative(u_t, x2, order=2)
    residuo = u_xx + u_yy +  32 * ((1 - X2) * X2 + (1 - X1) * X1) 
    loss = torch.mean(residuo**2)
    return loss

# Usar una función de optimización para minimizar la función de error con respecto a los coeficientes
 
optimizer = torch.optim.LBFGS([initial_guess],
                                lr=1,
                                max_iter=10_000,
                                max_eval=10_000,
                                tolerance_grad=1e-6,
                                history_size=50,
                                tolerance_change=1.0 * np.finfo(float).eps,
                                line_search_fn=None)

def closure():
    optimizer.zero_grad()
    loss = error_function(initial_guess, leg_x, leg_y)
    print(f"Loss: {loss.item()}")
    loss.backward(retain_graph=True)
    return loss

# Realizar pasos de optimización
start_optimization = time.time()
optimizer.step(closure)
end_optimization = time.time()

# Coeficientes óptimos encontrados
optimal_coefficients = initial_guess.detach()

# Evaluar la aproximación de la serie de Legendre
start_evaluation = time.time()
approximation = evaluate_legendre_series(optimal_coefficients, leg_x, leg_y)
end_evaluation = time.time()

# Medir el tiempo de optimización y evaluación
time_optimization = end_optimization - start_optimization
time_evaluation = end_evaluation - start_evaluation

print(f"Tiempo de optimización: {time_optimization:.4f} segundos")
print(f"Tiempo de evaluación: {time_evaluation:.4f} segundos")

# Graficar el resultado usando contourf
plt.figure(figsize=(8, 6))
plt.contourf(X1.cpu().numpy(), X2.cpu().numpy(), approximation.cpu().numpy(), cmap='viridis')
plt.colorbar(label='Approximation')
plt.title('Approximation of f(x1, x2) using Legendre Series')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

Tiempo de precomputación de Legendre polynomials: 0.2525 segundos
Loss: 123.32942209909866
Loss: 1914.712837244726
Loss: 121.90145315504584


KeyboardInterrupt: 

In [ ]:
# Calcular el error en la predicción
prediction_error = error_function(optimal_coefficients, leg_x, leg_y)

print(f"Error en la predicción: {prediction_error.item()}")

Error en la predicción: 9.475960452509686e-09
